In [1]:
# this script helps calculate the data stored on a satellite over time as a function of the times it is recording data
# and the times it can downlink
#
# this script calculates the access times of two chains and write them to .txt files for use in a Matlab code
# to use this script requires some set-up of your relevant chains
# in the scenario built for this, the satellite only captures data when it is above land, and downlinks when it has access with ground stations
# 
#To use it this way: create constellations of your ground stations receivers and all land area targets imported using the Land shapefile. Chain between satellite transmitter and receiver constellation, and chain between satellite and land constellation
# To use it another way: create two chains, one for your Downlinking communications setup and the other for relevant chain of when you will collect data
#
# author: Kyle Schmauk
# Date: 8/11/2020

# Note you may need to change the scenario times, and name of chains within the code to adapt to your scenario

%config IPCompleter.greedy=True  #Ensures intellisense in Jupyter is on

#Import basic utilities
import datetime as dt
import numpy as np
import os

##Needed to interact with COM
from comtypes.client import CreateObject

In [2]:
##Start STK12 Application
#uiApp = CreateObject("STK12.Application")

## Connect to running instance of STK
from comtypes.client import GetActiveObject
uiApplication = GetActiveObject("STK12.Application")

In [3]:
import comtypes
from comtypes.client import CreateObject


In [4]:
uiApplication.Visible = True


uiApplication.UserControl = True

In [5]:
#Get the STK Object Model Root
root = uiApplication.Personality2

In [6]:
#Import the STKObjects and STKUtil COM libraries
from comtypes.gen import STKObjects
from comtypes.gen import STKUtil

In [7]:
from comtypes.client import gen_dir
print(gen_dir)
os.listdir(gen_dir)

C:\Users\kschmauk\AppData\Local\Continuum\anaconda3\lib\site-packages\comtypes\gen


['AgSTKGraphicsLib.py',
 'AgSTKVgtLib.py',
 'AgUiApplicationLib.py',
 'AgUiCoreLib.py',
 'stdole.py',
 'STKObjects.py',
 'STKUtil.py',
 '_00020430_0000_0000_C000_000000000046_0_2_0.py',
 '_04C3E93C_A86F_4EB6_ADC5_FB4C61CB1489_0_1_0.py',
 '_39FCFC4F_D14A_4630_BF0E_E7C530880E31_0_1_0.py',
 '_3BE3E949_CFB2_45D3_BDDE_0D8324DFC3E2_0_1_0.py',
 '_51602B4D_7756_4200_8561_D2D287D520E2_0_1_0.py',
 '_C5056BCE_6206_4ECE_BF04_1C440C0F557C_0_1_0.py',
 '_CBA9FFF0_F6A6_4E53_9A4A_738832A1C9B6_0_1_0.py',
 '__init__.py',
 '__pycache__']

In [8]:
######## Replace the StartTime and StopTime with your relevant scenario analysis times ###############
scenario = root.CurrentScenario

scenario2 = scenario.QueryInterface(STKObjects.IAgScenario)

scenario2.StartTime = "17 Jul 2020 16:00:00.000"
scenario2.StopTime = "18 Jul 2020 16:00:00.000"

In [9]:
type(scenario)

comtypes.POINTER(IAgStkObject)

In [10]:
dir(scenario)

['AccessConstraints',
 'AddRef',
 'CentralBodyName',
 'Children',
 'ClassName',
 'ClassType',
 'CopyObject',
 'CreateOnePointAccess',
 'DataProviders',
 'Export',
 'GetAccess',
 'GetAccessToObject',
 'HasChildren',
 'InstanceName',
 'IsAccessSupported',
 'IsObjectCoverageSupported',
 'IsVgtSupported',
 'LongDescription',
 'Metadata',
 'ObjectCoverage',
 'ObjectFiles',
 'Parent',
 'Path',
 'QueryInterface',
 'Release',
 'Root',
 'ShortDescription',
 'Unload',
 'Vgt',
 '_AddRef',
 '_IAgStkObject__com_CopyObject',
 '_IAgStkObject__com_CreateOnePointAccess',
 '_IAgStkObject__com_Export',
 '_IAgStkObject__com_GetAccess',
 '_IAgStkObject__com_GetAccessToObject',
 '_IAgStkObject__com_IsAccessSupported',
 '_IAgStkObject__com_IsObjectCoverageSupported',
 '_IAgStkObject__com_IsVgtSupported',
 '_IAgStkObject__com_Unload',
 '_IAgStkObject__com__get_AccessConstraints',
 '_IAgStkObject__com__get_CentralBodyName',
 '_IAgStkObject__com__get_Children',
 '_IAgStkObject__com__get_ClassName',
 '_IAgStkObj

In [16]:
######## This code uses a 90 degree elevation angle constraint of all the land area targets to create a distinction when a satellite is directly over land versus over water
root.ExecuteCommand('SetConstraint */AreaTarget/* ElevationAngle 90.0')

root.UnitPreferences.Item('DateFormat').SetCurrentUnit('UTCG')

0

In [17]:
############ For preexisting Chain Elements #########

ChainList = scenario.Children.GetElements(STKObjects.eChain)
for num in range(0,len(ChainList)):
    print(ChainList[num].InstanceName)

Over_Land
DownLink


In [18]:
############

ChainEOIR = ChainList[0]
ChainEOIR2 = ChainEOIR.QueryInterface(STKObjects.IAgChain)
ChainEOIR2.AutoRecompute = False
ChainEOIR2.EnableLightTimeDelay = False
ChainEOIR2.TimeConvergence = 0.001
ChainEOIR2.DataSaveMode = 2
   
ChainEOIR2.SetTimePeriodType(2)
chainUserTimePeriod = ChainEOIR2.TimePeriod
chainUserTimePeriod2 = chainUserTimePeriod.QueryInterface(STKObjects.IAgChUserSpecifiedTimePeriod)
chainUserTimePeriod2.TimeInterval.SetStartTimeAndDuration(scenario2.StartTime,scenario2.StopTime)

ChainEOIR2.ComputeAccess()

eoirAccess = ChainEOIR.DataProviders.Item("Complete Access").QueryInterface(STKObjects.IAgDataPrvInterval).Exec(scenario2.StartTime,scenario2.StopTime)

root.UnitPreferences.Item('DateFormat').SetCurrentUnit('EpSec')

elEOIR = eoirAccess.DataSets.ElementNames
numRowsEOIR = eoirAccess.DataSets.RowCount
print(elEOIR)
print("\nEOIR chain access data")
with open("EOIRAccess.txt", "w") as dataFile:
    dataFile.write(f"{elEOIR[0]},{elEOIR[1]},{elEOIR[2]},{elEOIR[3]}\n")
    print(f"{elEOIR[0]},{elEOIR[1]},{elEOIR[2]},{elEOIR[3]}")
    
    for row in range(numRowsEOIR):
        rowDataEOIR = eoirAccess.DataSets.GetRow(row)
        dataFile.write(f"{rowDataEOIR[0]},{rowDataEOIR[1]},{rowDataEOIR[2]},{rowDataEOIR[3]}\n")
        print(f"{rowDataEOIR[0]},{rowDataEOIR[1]},{rowDataEOIR[2]},{rowDataEOIR[3]}")

('Access Number', 'Start Time', 'Stop Time', 'Duration')

EOIR chain access data
Access Number,Start Time,Stop Time,Duration
1,1121.834330196,1453.954654279,332.1203240834582
2,1455.066363733,1549.219636573,94.15327283950273
3,1604.009827365,1670.18721889,66.17739152475224
4,1670.448750889,1672.090301509,1.6415506206533337
5,2566.169090062,2854.247467345,288.0783772830332
6,4514.007036749,4517.053814173,3.046777424116044
7,4520.36089063,4523.102778909,2.7418882793253943
8,4532.878547972,4537.786403278,4.907855306051715
9,5233.65800782,5525.594980123,291.9369723025884
10,5806.392160535,5809.163765008,2.7716044728895213
11,7046.360122088,7472.549122564,426.1890004754996
12,7474.540022512,7618.588383432,144.0483609201492
13,7641.607036401,7648.654653881,7.047617479419387
14,7660.978787114,7671.157346489,10.178559374846373
15,7691.676666304,7697.53183012,5.855163815283959
16,7715.223628379,7964.472175599,249.24854721945303
17,7992.228333253,8019.19206087,26.963727616533106
18,10362.7453184

In [19]:
root.UnitPreferences.Item('DateFormat').SetCurrentUnit('UTCG')

ChainComm = ChainList[1]
ChainComm2 = ChainComm.QueryInterface(STKObjects.IAgChain)
ChainComm2.AutoRecompute = False
ChainComm2.EnableLightTimeDelay = False
ChainComm2.TimeConvergence = 0.001
ChainComm2.DataSaveMode = 2
   
ChainComm2.SetTimePeriodType(2)
chainUserTimePeriod = ChainComm2.TimePeriod
chainUserTimePeriod2 = chainUserTimePeriod.QueryInterface(STKObjects.IAgChUserSpecifiedTimePeriod)
chainUserTimePeriod2.TimeInterval.SetStartTimeAndDuration(scenario2.StartTime,scenario2.StopTime)

ChainComm2.ComputeAccess()

commAccess = ChainComm.DataProviders.Item("Complete Access").QueryInterface(STKObjects.IAgDataPrvInterval).Exec(scenario2.StartTime,scenario2.StopTime)

root.UnitPreferences.Item('DateFormat').SetCurrentUnit('EpSec')

elComm = commAccess.DataSets.ElementNames
numRowsComm = commAccess.DataSets.RowCount
print(elComm)
print("\nComm chain access data")
with open("CommAccess.txt", "w") as dataFile:
    dataFile.write(f"{elComm[0]},{elComm[1]},{elComm[2]},{elComm[3]}\n")
    print(f"{elComm[0]},{elComm[1]},{elComm[2]},{elComm[3]}")
    
    for row in range(numRowsComm):
        rowDataComm = commAccess.DataSets.GetRow(row)
        dataFile.write(f"{rowDataComm[0]},{rowDataComm[1]},{rowDataComm[2]},{rowDataComm[3]}\n")
        print(f"{rowDataComm[0]},{rowDataComm[1]},{rowDataComm[2]},{rowDataComm[3]}")

('Access Number', 'Start Time', 'Stop Time', 'Duration')

Comm chain access data
Access Number,Start Time,Stop Time,Duration
1,3823.319734163,4485.999803994,662.6800698309971
2,9754.155723539,10553.715674667,799.5599511277342
3,15765.018438914,16615.188932637,850.1704937228678
4,21815.648171137,22676.507743639,860.8595725020023
5,23492.436256261,24288.058682342,795.6224260806557
6,27877.501628348,28722.878499017,845.3768706686606
7,29579.334588923,30351.98818344,772.653594516385
8,33938.949003719,34723.582370895,784.6333671763568
9,36591.215694736,36967.118249021,375.90255428543605
10,40010.392128221,40635.958493959,625.5663657381883
11,42130.220549981,43068.871362779,938.6508127980051
12,46220.015933621,46309.720255593,89.70432197183254
13,48038.356639554,49109.667457164,1071.3108176101305
14,54111.013557628,55142.162666388,1031.1491087594259
15,60263.828677605,61218.150504119,954.3218265148462
16,61573.987004761,62216.214921395,642.2279166339868
17,66391.504245574,67283.80251551,892.

86400.0